In [150]:
!pip install wordfreq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 16.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 725.2 kB/s eta 0:00:00 MB/s eta 0:00:01

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [158]:
import spacy
import numpy as np
from wordfreq import zipf_frequency

Vectors are only available in larger models, so let's download that first:

In [62]:
!python3 -m spacy download en_core_web_md

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.7 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python3.10 -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [138]:
nlp = spacy.load('en_core_web_lg')

s = "It's not about the money (only $20.15), it's about sending a message :). 🚀💎🙌"
doc = nlp(s)


In [139]:
token = doc[1]
token

's

In [140]:
def get_vocab_id(word: str):
    return nlp.vocab.strings[word]

def get_vector(word: str):
    return nlp.vocab.vectors[vocab_id(word)]

def get_token(word: str):
    return nlp(word)[0]
    

## Finding similarities

In [141]:
get_token("man").similarity(get_token("king"))

0.41661593317985535

In [142]:
tokens = nlp(u'cat lion pet')

for t1 in tokens:
    for t2 in tokens:
        print(t1.text,t2.text,t1.similarity(t2))

cat cat 1.0
cat lion 0.3854507803916931
cat pet 0.732966423034668
lion cat 0.3854507803916931
lion lion 1.0
lion pet 0.20031583309173584
pet cat 0.732966423034668
pet lion 0.20031583309173584
pet pet 1.0


In [143]:
tokens = nlp(u'castle king student pub')

for t1 in tokens:
    for t2 in tokens:
        print(f"{t1.text}, {t2.text}, {t1.similarity(t2):.3f}")

castle, castle, 1.000
castle, king, 0.494
castle, student, -0.021
castle, pub, 0.305
king, castle, 0.494
king, king, 1.000
king, student, 0.201
king, pub, 0.014
student, castle, -0.021
student, king, 0.201
student, student, 1.000
student, pub, -0.062
pub, castle, 0.305
pub, king, 0.014
pub, student, -0.062
pub, pub, 1.000


## Find most similar words to a given vector
Documentation https://spacy.io/api/vectors#most_similar

In [206]:
def find_most_similar(vec: np.ndarray):
    # vec = vector(word)

    vocab_ids = nlp.vocab.vectors.most_similar(np.asarray([vec]), n=100)
    words =  [nlp.vocab.strings[w] for w in vocab_ids[0][0]]

    return [w for w in words if get_token(w).is_alpha & (zipf_frequency(w, "en", wordlist='small', minimum=1)> 3)][0:20]



In [207]:
find_most_similar(get_vector("doctor"))

['doctor',
 'physician',
 'psychiatrist',
 'doctors',
 'dentist',
 'nurse',
 'pharmacist',
 'pediatrician',
 'surgeon',
 'proctor',
 'dermatologist',
 'veterinarian',
 'midwife',
 'psychiatrists',
 'therapist',
 'neurologist',
 'clinic',
 'medic',
 'Pediatrician',
 'physicians']

In [208]:
find_most_similar(get_vector("doctor") - get_vector("man") + get_vector("woman"))

['doctor',
 'pediatrician',
 'nurse',
 'midwife',
 'physician',
 'Pediatrician',
 'dermatologist',
 'dentist',
 'therapist',
 'Dermatologist',
 'clinic',
 'pharmacist',
 'doctors',
 'Midwife',
 'pediatrics',
 'psychiatrist',
 'veterinarian',
 'pediatric',
 'neurologist',
 'Physician']

In [209]:
find_most_similar(get_vector("king") - get_vector("man") + get_vector("girl"))

['king',
 'princess',
 'princesses',
 'princes',
 'prince',
 'kings',
 'queen',
 'consort',
 'Mcqueen',
 'mcqueen',
 'monarch',
 'ruler',
 'rulers',
 'Princesses',
 'thrones',
 'kingdom',
 'monarchs',
 'throne',
 'kingdoms',
 'royal']

In [213]:
find_most_similar(get_vector("castle") - get_vector("royalty") + get_vector("student"))

['student',
 'campus',
 'school',
 'dormitory',
 'university',
 'castle',
 'students',
 'classmate',
 'pupil',
 'teacher',
 'classroom',
 'headmaster',
 'undergraduate',
 'college',
 'Dormitory',
 'undergraduates',
 'gymnasium',
 'schoolboy',
 'pupils',
 'highschool']


You could, for instance, simply add two vectors together to get a new vector, and search for the nearest word. The results, as we have seen, often made a shocking amount of sense:
Czech + currency = korunaVietnam + capital = Hanoi
German + airlines = Lufthansa
French + actress = Juliette Binoche

And you could subtract words, too. This meant—incredibly—you could produce “analogies” by getting the “difference” between two words and then “adding” it to a third.66
These analogies suggested that the embeddings had captured geography:

Berlin − Germany + Japan = Tokyo

And grammar:
bigger − big + cold = colder

And cuisine:
sushi − Japan + Germany = bratwurst

And science:
Cu − copper + gold = Au

And tech:
Windows − Microsoft + Google = Android

And sports:
Montreal Canadiens − Montreal + Toronto = Toronto Maple Leafs67


## Further reading
- Library for embedding exploration: https://github.com/koaning/whatlies/